<a href="https://colab.research.google.com/github/vsoares1998/Big-Data/blob/main/C%C3%B3pia_de_BigQuery_running_queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BigQuery - running queries


Autores: 

Tersa Romão: up201603999

Nuno Nunes: up201606441 

Vasco Soares: up201604364

**Google Cloud Project**: bdcc-project-201603999





## Get access to Google Cloud from the notebook



In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Set project id

In [12]:
PROJECT_ID='bdcc-project-201603999' #@param {type: "string"}

### Authenticate

In [13]:
from google.colab import auth
auth.authenticate_user()
#!gcloud config set project {PROJECT_ID}

### /**image_search_multiple**

Na query "image search multiple" são feitas junções das tabelas Classes e Labels usando o Label como elemento em comum, foi também usado o UNNEST para podermos criar um array com elesmentos a ter em conta no where. No select usamos o ARRAY_AGG de forma a conseguirmos obter um conjunto de elementos.

In [14]:
%%bigquery --project $PROJECT_ID


SELECT image.ImageId, ARRAY_AGG(classes.Description) as classes 
FROM `bdcc22project.openimages.classes` as classes 
JOIN `bdcc22project.openimages.image_labels` as image USING(Label), UNNEST(["Dog", "Cat"]) as animals 
WHERE classes.description like animals 
GROUP BY 1 order by array_length(classes) desc limit 10


,ImageId,classes
0,60ddbb1c45154865,"[Cat, Dog]"
1,c0303c533156ff7c,"[Cat, Dog]"
2,93a33e91cf0f5486,"[Cat, Dog]"
3,0ff037d4305d0f10,"[Cat, Dog]"
4,a801eed54a988ca1,"[Cat, Dog]"
5,d5a19e615dc779ef,"[Cat, Dog]"
6,ce26d0f13deb5842,"[Cat, Dog]"
7,00d9419e2faa59fa,[Cat]
8,017c776a67956dd9,[Cat]
9,018376cd9ec14643,[Cat]


### /**relations**

Na query "relations" são feitas junções de tres tabelas (classes,relations,tester) usando como elemento comum as labels mas tendo em conta a sua associação com cada tabela.

In [15]:
%%bigquery --project $PROJECT_ID
        
SELECT relations.Relation, COUNT(*) AS NumImages
        FROM `bdcc22project.openimages.classes` as classes
        JOIN `bdcc22project.openimages.relations` as relations ON relations.label1 = classes.label
        LEFT JOIN `bdcc22project.openimages.classes` as tester ON tester.Label = relations.Label2
        GROUP BY Relation
        ORDER BY Relation 


  

,Relation,NumImages
0,at,861
1,hits,54
2,holds,658
3,inside_of,161
4,interacts_with,272
5,on,550
6,plays,175
7,under,3
8,wears,34


### /**relations_search**

Na query "relations search" são feitas junções de tres tabelas (classes,relations,tester) como na query anterior e usando como elemento comum as labels mas tendo em conta a sua associação com cada tabela.

In [16]:
%%bigquery --project $PROJECT_ID
SELECT relations.ImageId, classes.Description, relations.Relation, tester.Description
                FROM `bdcc22project.openimages.classes` as classes
                JOIN `bdcc22project.openimages.relations` as relations ON relations.label1 = classes.label
                LEFT JOIN `bdcc22project.openimages.classes` as tester ON tester.Label = relations.Label2
                WHERE relations.Relation LIKE 'plays' 
                and classes.Description LIKE 'Girl' 
                and tester.Description LIKE 'Violin'  
                order by 1
                limit 10

,ImageId,Description,Relation,Description_1
0,0041c772f8b9aef6,Girl,plays,Violin
1,708228e99ab1429c,Girl,plays,Violin
2,72a9928a7e27d7a8,Girl,plays,Violin
3,82c39a4b066fe331,Girl,plays,Violin
4,8dc418a8b5d412a5,Girl,plays,Violin
5,b8a69b4b027a98c1,Girl,plays,Violin
6,e0bc92015fe1a21c,Girl,plays,Violin
7,e563022367b0fe06,Girl,plays,Violin


### /**image_info**


Na query "Image Info" são feitas as junçoes de duas tabelas (classes e labels)
usando label como referência.


In [17]:
%%bigquery --project $PROJECT_ID
SELECT Description
        FROM `bdcc22project.openimages.classes`
        JOIN `bdcc22project.openimages.image_labels` USING(Label)
        WHERE ImageId = '0041c772f8b9aef6' 
        ORDER BY Description

,Description
0,Clothing
1,Girl
2,Human eye
3,Human hair
4,Human hand
5,Human mouth
6,Human nose
7,Violin


In [19]:
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic


Reading package lists... Done
Building dependency tree       
Reading state information... Done
texlive-fonts-recommended is already the newest version (2017.20180305-1).
texlive-plain-generic is already the newest version (2017.20180305-2).
texlive-xetex is already the newest version (2017.20180305-1).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
